In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [60]:
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import class_weight
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
import pickle
import nltk
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Input
from keras.models import Sequential, Model
from keras import optimizers, regularizers, initializers
from keras.callbacks import ModelCheckpoint, Callback
from keras import backend as K
from keras.optimizers import Adam
import tensorflow as tf
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
NCA1 = 100
NCA2 = 100
DROPRATE = 0.2
EP = 500
BATCH_SIZE = 256
VAL_RATIO = 0.1
TEST_RATIO = 0.1

## Loading DrugBank dataset

In [111]:
db_df= pd.read_csv('../input/dbslim_descriptors/dbslim_df_revised.csv', encoding='cp1252')
print(db_df.shape)
db_df.head()

(1242, 33)


,type,drugbank_id,name,description,pharmacodynamics,mechanism-of-action,toxicity,metabolism,absorption,half-life,protein-binding,route-of-elimination,clearance,groups,targets,enzymes,atc_codes,categories,inchi,inchikey,SMILES,logS,logP,Water_Solubility,Melting_Point,Boiling_Point,Hydrophobicity,Isoelectric_Point,caco2_Permeability,pKa,Radioactivity,aliases,smiles
0,small molecule,DB00006,Bivalirudin,Bivalirudin is a synthetic 20 residue peptide ...,Bivalirudin directly and reversibly inhibits t...,Inhibits the action of thrombin by binding bot...,"Based on a study by Gleason et al., the no-obs...",80% proteolytic cleavage,"Following intravenous administration, bivaliru...",* Normal renal function: 25 min (in normal con...,"Other than thrombin and red blood cells, bival...",Bivalirudin is cleared from plasma by a combin...,* 3.4 mL/min/kg [Normal renal function]\r\r\n*...,approved|investigational,\r\n,\r\n,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,OIRCOABEOLEUMC-GEJPAHFPSA-N,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n |Angiomax|Bivalirudin|Bivalirudin fo...,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,small molecule,DB00014,Goserelin,"Goserelin is a synthetic hormone. In men, it s...",The pharmacokinetics of goserelin have been de...,Goserelin is a synthetic decapeptide analogue ...,No experience of overdosage from clinical trials.,Hepatic,"Inactive orally, rapidly absorbed following su...",4-5 hours,27.3%,Clearance of goserelin following subcutaneous ...,* 121 +/- 42.4 mL/min [prostate cancer with 10...,approved,\r\n |\r\n,NaN,L02AE03,"Adrenal Cortex Hormones|Amino Acids, Peptides,...",InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,BLCLNMBMMGCOAS-URPVMXJPSA-N,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n |Goserelin|Zoladex|Zoladex LA,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,small molecule,DB00035,Desmopressin,"Desmopressin (dDAVP), a synthetic analogue of ...","By mimicking the actions of endogenous ADH, de...",Upon binding of desmopressin to V2 receptors i...,Intravenous TDLo in humans is reported to be 0...,"In vitro, in human liver microsome preparation...",Following nasal spray administration of 0.83 m...,Following an intranasal dose of 1.66 mcg of de...,Following radioiodination (125I) in the N-term...,Desmopressin is mainly excreted in the urine. ...,NaN,approved,\r\n |\r\n |\r\n,\r\n |\r\n,H01BA02,"Agents that produce hypertension|Amino Acids, ...",InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,NFLWUMRGJYTJIN-NXBWRCJVSA-N,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r\n |DDAVP Rhinal Tube|Ddavp|Ddavp Inj 4...,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
3,small molecule,DB00050,Cetrorelix,Cetrorelix is a man-made hormone that blocks t...,Cetrorelix is a synthetic decapeptide with gon...,Cetrorelix binds to the gonadotropin releasing...,NaN,"In in vitro studies, cetrorelix was stable aga...",Rapidly absorbed following subcutaneous inject...,~62.8 hours,86%,Following subcutaneous administration of 10 mg...,* 1.28 ml/min·kg [adult healthy female with 3 ...,approved|investigational,\r\n |\r\n,NaN,H01CC02,"Amino Acids, Peptides, and Proteins|Anti-Gonad...",InChI=1S/C70H92ClN17O14/c1-39(2)31-52(61(94)82...,SBNPWPIBESPSIF-MHWMIDJBSA-N,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cetrorelix|Cetrotide,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...
4,small molecule,DB00080,Daptomycin,Daptomycin is a lipopeptide antibiotic that ki...,Daptomycin is a 13 member amino acid cyclic li...,Daptomycin appears to bind or insert into the ...,NaN,Minor amounts of three oxidative metabolites a...,Generally exhibits linear and time-independent...,Half-life elimination: 8-9 hours (up to 28 hou...,Daptomycin is reversibly bound to human plasma...,Daptomycin is

In [ ]:
db_df.columns

In [ ]:
a = set(db_df['atc_codes'].values)
len(a)

In [ ]:
db_description_df = db_df['description']
print(db_description_df.shape)
db_description_df.head()

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()

In [95]:
db_description = []
for desc in db_description_df.values:
    tok = nltk.word_tokenize(str(desc))
    #tok = [stemmer.stem(single.lower()) for single in tok]
    tok = set(tok)
    db_description.append(tok)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [136]:
cls_moa = ['inhibitor','blocker', 'agonist','antagonist']

cls_theraputic = ['analgesic','antibiotic', 'anticoagulant', 'antidepressant', 
                  'anticancer', 'antiepileptic', 'antipsychotic', 'antiviral',
                  'sedative', 'antidiabetic']

cls_theraputic_ = ['analgesic','anti-biotic', 'anti-coagulant', 'anti-depressant', 
                  'anti-cancer', 'anti-epileptic', 'anti-psychotic', 'anti-viral',
                  'sedative', 'anti-diabetic']

In [97]:
for i in range(len(cls_moa)):
    cls_moa[i] = set(nltk.word_tokenize(str(cls_moa[i])))

In [140]:
for i in range(len(cls_therapeutic)):
    cls_therapeutic[i] = set(nltk.word_tokenize(str(cls_therapeutic[i])))

In [139]:
for i in range(len(cls_therapeutic_)):
    cls_therapeutic_[i] = set(nltk.word_tokenize(str(cls_therapeutic_[i])))

In [109]:
db_moa_df = db_df
for i in tqdm(range(len(db_description))):
    if cls_moa[0].issubset(db_description[i]):
        db_moa_df.loc[i,'inhibitor'] = 1
    else:
        db_moa_df.loc[i,'inhibitor'] = 0
    
    if cls_moa[1].issubset(db_description[i]):
        db_moa_df.loc[i,'blocker'] = 1
    else:
        db_moa_df.loc[i,'blocker'] = 0
   
    if cls_moa[2].issubset(db_description[i]):
        db_moa_df.loc[i,'agonist'] = 1
    else:
        db_moa_df.loc[i,'agonist'] = 0
    
    if cls_moa[3].issubset(db_description[i]):
        db_moa_df.loc[i,'antagonist'] = 1
    else:
        db_moa_df.loc[i,'antagonist'] = 0

100%|██████████| 1242/1242 [00:02<00:00, 479.73it/s]


In [113]:
db_target_df.to_csv('db_target_df.csv',index=False)

In [141]:
db_therapeutic_df = db_df
for i in tqdm(range(len(db_description))):
    if cls_therapeutic[0].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'analgesic'] = 1
    else:
        db_therapeutic_df.loc[i,'analgesic'] = 0
    
    if cls_therapeutic[1].issubset(db_description[i]) or cls_therapeutic_[1].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'antibiotic'] = 1
    else:
        db_therapeutic_df.loc[i,'antibiotic'] = 0
   
    if cls_therapeutic[2].issubset(db_description[i]) or cls_therapeutic_[2].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'anticoagulant'] = 1
    else:
        db_therapeutic_df.loc[i,'anticoagulant'] = 0
    
    if cls_therapeutic[3].issubset(db_description[i]) or cls_therapeutic_[3].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'antidepressant'] = 1
    else:
        db_therapeutic_df.loc[i,'antidepressant'] = 0
        
    if cls_therapeutic[4].issubset(db_description[i]) or cls_therapeutic_[4].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'anticancer'] = 1
    else:
        db_therapeutic_df.loc[i,'anticancer'] = 0
        
    if cls_therapeutic[5].issubset(db_description[i]) or cls_therapeutic_[5].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'antiepileptic'] = 1
    else:
        db_therapeutic_df.loc[i,'antiepileptic'] = 0
        
    if cls_therapeutic[6].issubset(db_description[i]) or cls_therapeutic_[6].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'antipsychotic'] = 1
    else:
        db_therapeutic_df.loc[i,'antipsychotic'] = 0
        
    if cls_therapeutic[7].issubset(db_description[i]) or cls_therapeutic_[7].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'antiviral'] = 1
    else:
        db_therapeutic_df.loc[i,'antiviral'] = 0
        
    if cls_therapeutic[8].issubset(db_description[i]) or cls_therapeutic_[8].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'sedative'] = 1
    else:
        db_therapeutic_df.loc[i,'sedative'] = 0
        
    if cls_therapeutic[9].issubset(db_description[i]) or cls_therapeutic_[9].issubset(db_description[i]):
        db_therapeutic_df.loc[i,'antidiabetic'] = 1
    else:
        db_therapeutic_df.loc[i,'antidiabetic'] = 0

100%|██████████| 1242/1242 [00:05<00:00, 240.17it/s]


In [145]:
db_theraputic_df['antipsychotic'].value_counts()

0.0    1214
1.0      28
Name: antipsychotic, dtype: int64

In [104]:
db_theraputic_df.to_csv('db_theraputic_df.csv',index=False)

True